In [1]:
! pip install langchain_openai langchain -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.60
    Uninstalling langchain-core-0.3.60:
      Successfully uninstalled langchain-core-0.3.60


LangChain 表达式语言 （LCEL） 是一种链接 LangChain 组件的声明性方式。它能够让你以管道（pipe）的形式，线性的调用大模型和处理任务。

LCEL详细文档： https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

例如我们有一个需求，先调用大模型拿到原始结果 **aiRes**，再获取结果中的字符串内容 **str_parser**。

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

import os
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="qwen2.5-72b-instruct")

str_parser = StrOutputParser()

aiRes = llm.invoke("你好")

res = str_parser.invoke(aiRes)
res

'你好！有什么我可以帮助你的吗？'

The content property describes the content of the message. This can be a few different things:


*   A string (most models deal with this type of content)
*   A List of dictionaries (this is used for multimodal input, where the dictionary contains information about that input type and that input location)

In [3]:
aiRes

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 9, 'total_tokens': 17, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-72b-instruct', 'system_fingerprint': None, 'id': 'chatcmpl-0bd9040a-fb9c-9fa0-a5a0-f2285297fc9d', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--213ab6b4-9ff9-41e5-9ce6-4cd380316053-0', usage_metadata={'input_tokens': 9, 'output_tokens': 8, 'total_tokens': 17, 'input_token_details': {}, 'output_token_details': {}})

In [4]:
res = str_parser.invoke(aiRes)
res

'你好！有什么我可以帮助你的吗？'

如果我们使用LCEL的形式，我们可以这样做：



``` python
chain = llm | StrOutputParser()
```
官网指导文档：
https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

import os
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="qwen2.5-72b-instruct")

chain = llm | StrOutputParser()

res = chain.invoke("你好")
res

'你好！有什么可以帮助你的吗？'

# Runnable interface
为了尽可能轻松地创建自定义链，langchain提供了 “Runnable” 协议。你也应该尽力让链上的每一个方法都是一个Runnable对象。许多 LangChain 组件都实现了 Runnable 协议，包括聊天模型、LLMs、输出解析器、检索器、提示模板等。


# String PromptTemplate
* These prompt templates are used to format **a single string**, and generally are used for simpler inputs.

For example, a common way to construct and use a PromptTemplate is as follows:

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt_template.invoke({"topic": "cats"})

StringPromptValue(text='Tell me a joke about cats')


Prompt Templates take as input a dictionary, where each key represents a variable in the prompt template to fill in.

Prompt Templates output a **PromptValue**. This PromptValue can be passed to an LLM or a ChatModel, and can also be cast to a string or a list of messages.

The reason this PromptValue exists is to make it easy to switch between strings and messages.
```
StringPromptValue(text='Tell me a joke about cats')
```




# ChatPromptTemplate

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个智能助手"),
    ("user", "告诉我一个笑话，关于 {topic}")
])

template = prompt_template.invoke({"topic": "小猫"})
print(template)



messages=[SystemMessage(content='你是一个智能助手', additional_kwargs={}, response_metadata={}), HumanMessage(content='告诉我一个笑话，关于 小猫', additional_kwargs={}, response_metadata={})]


In the above example, this ChatPromptTemplate will construct two messages when called. The first is a system message, that has no variables to format. The second is a HumanMessage, and will be formatted by the topic variable the user passes in.

# LLM
然后我们调用大模型，运行流程如下：
- 调用chain.invoke("小鸡")，先基于小鸡生成了模板[SystemMessage(content='你是一个智能助手'), HumanMessage(content='告诉我一个笑话，关于 小鸡')]
- 然后将模板传入大模型llm，获得AIMessage对象
- 把AIMessage传给StrOutputParser，生成字符串结果

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

import os
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(model="qwen2.5-72b-instruct")

chain = prompt_template | llm | StrOutputParser()

res = chain.invoke({"topic": "小鸡"})
print(res)


小鸡为什么跑得那么快？因为有只鹰叫“妈妈”！


In [11]:
from langchain.schema import AIMessage
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()

ai_message = AIMessage(content="这是一个AI消息")
print(ai_message)

res = str_parser.invoke(ai_message)
print(res)


content='这是一个AI消息' additional_kwargs={} response_metadata={}
这是一个AI消息
